In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import EarlyStoppingCallback,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from textstat.textstat import textstatistics
import pandas as pd
import language_tool_python
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df= pd.read_json('datasets/subtaskA_train_monolingual.jsonl', lines=True)
test_df= pd.read_json('datasets/subtaskA_dev_monolingual.jsonl', lines=True)
df=df.sample(20000)
print(df['label'].value_counts())

label
0    10600
1     9400
Name: count, dtype: int64


In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize the grammar checker
tool = language_tool_python.LanguageTool('en-US')

def extract_text_features(text):
    # Tokenize the text
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    word_count = len(words)
    sentence_count = len(sentences)

    # Lexical diversity
    unique_words = set(words)
    lexical_diversity = len(unique_words) / word_count

    # Average sentence length
    avg_sentence_length = word_count / sentence_count

    # Average word length
    avg_word_length = sum(len(word) for word in words) / word_count

    # Lexical density
    tags = nltk.pos_tag(words)
    content_words = [word for word, tag in tags if tag.startswith(('N', 'V', 'J', 'R'))]
    lexical_density = len(content_words) / word_count

    # Readability scores
    flesch_reading_ease = textstatistics().flesch_reading_ease(text)
    fog_index = textstatistics().gunning_fog(text)

    # Grammatical errors
    matches = tool.check(text)
    grammatical_errors = len(matches)

    # Create a DataFrame
    return pd.Series({
        'word_count': word_count,
        'sentence_count': sentence_count,
        'lexical_diversity': lexical_diversity,
        'avg_sentence_length': avg_sentence_length,
        'avg_word_length': avg_word_length,
        'lexical_density': lexical_density,
        'flesch_reading_ease': flesch_reading_ease,
        'gunning_fog_index': fog_index,
        'grammatical_errors': grammatical_errors
    })

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
from tqdm import tqdm
tqdm.pandas()
features_df = df['text'].progress_apply(extract_text_features)
features_df_test = test_df['text'].progress_apply(extract_text_features)

100%|██████████| 5000/5000 [09:31<00:00,  8.74it/s]  


In [5]:
features_df['label']=df['label']
features_df_test['label']=test_df['label']

features_df.to_csv('datasets/features_df.csv', index=False)
features_df_test.to_csv('datasets/features_df_test.csv', index=False)

features_df

,word_count,sentence_count,lexical_diversity,avg_sentence_length,avg_word_length,lexical_density,flesch_reading_ease,gunning_fog_index,grammatical_errors,label
15708,289.0,14.0,0.435986,20.642857,4.830450,0.584775,53.00,10.73,0.0,1
69578,337.0,11.0,0.403561,30.636364,4.112760,0.492582,68.26,7.06,4.0,0
45801,250.0,13.0,0.420000,19.230769,4.844000,0.580000,53.61,10.71,4.0,1
60067,236.0,17.0,0.504237,13.882353,3.542373,0.491525,92.83,6.75,5.0,0
18761,293.0,15.0,0.447099,19.533333,4.177474,0.529010,71.44,8.27,2.0,1
...,...,...,...,...,...,...,...,...,...,...
45531,213.0,7.0,0.539906,30.428571,5.281690,0.572770,25.53,18.04,3.0,1
101303,235.0,12.0,0.544681,19.583333,4.080851,0.548936,70.73,10.21,1.0,0
6409,253.0,14.0,0.557312,18.071429,4.332016,0.521739,64.30,9.32,13.0,1
79749,2631.0,81.0,0.271380,32.481481,4.752566,0.564804,42.14,13.46,103.0,0


In [6]:
complete_df=features_df
complete_df_test=features_df_test

features_df = features_df.drop('label', axis=1)
features_df_test = features_df_test.drop('label', axis=1)

def getMetrics(predicted_labels, true_labels):
    # Ensure the labels are numpy arrays
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)

    # Compute metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')[2]
    precision = precision_score(true_labels, predicted_labels, average='binary')
    recall = recall_score(true_labels, predicted_labels, average='binary')
    auc = roc_auc_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)

    # Create a dictionary of metrics
    metrics = {
        'f1': f1,
        'confusion_matrix': cm.tolist(),
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'auc': auc,
    }

    return metrics

X_train=features_df
y_train=df['label']
X_test=features_df_test
y_test=test_df['label']

# Generate all possible combinations of features
feature_combinations = []
for r in range(1, len(X_train.columns) + 1):
    feature_combinations.extend(combinations(X_train.columns, r))

# Iterate over combinations, train models, and evaluate
best_f1 = 0
best_metrics = None
best_combination = None

for combo in tqdm(feature_combinations):
    # Select features for this combination
    X_train_combo = X_train[list(combo)]
    X_test_combo = X_test[list(combo)]

    # Train the Random Forest model
    rf = RandomForestClassifier()
    rf.fit(X_train_combo, y_train)

    # Predict on the test set
    predicted_labels = rf.predict(X_test_combo)

    # Evaluate the model
    metrics = getMetrics(predicted_labels, y_test)
    
    print(f"Combination: {combo}, F1 Score: {metrics['f1']}")

    # Check if this is the best model so far
    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        best_metrics = metrics
        best_combination = combo

# Return the best F1 score, the corresponding combination of features, and the full metrics
best_f1, best_combination, best_metrics

  0%|          | 1/511 [00:01<11:05,  1.30s/it]

Combination: ('word_count',), F1 Score: 0.5355096935584741


  0%|          | 2/511 [00:01<06:24,  1.32it/s]

Combination: ('sentence_count',), F1 Score: 0.5115730960102902


  1%|          | 3/511 [00:07<26:51,  3.17s/it]

...

Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density'), F1 Score: 0.4140601149010574


 51%|█████▏    | 262/511 [19:44<21:53,  5.28s/it]

Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.4332408074190121


 51%|█████▏    | 262/511 [19:47<18:48,  4.53s/it]


KeyboardInterrupt: 

In [7]:
result='''
  0%|          | 1/511 [00:01<11:05,  1.30s/it]
Combination: ('word_count',), F1 Score: 0.5355096935584741
  0%|          | 2/511 [00:01<06:24,  1.32it/s]
Combination: ('sentence_count',), F1 Score: 0.5115730960102902
  1%|          | 3/511 [00:07<26:51,  3.17s/it]
Combination: ('lexical_diversity',), F1 Score: 0.4258323676324568
  1%|          | 4/511 [00:11<28:33,  3.38s/it]
Combination: ('avg_sentence_length',), F1 Score: 0.540274716778604
  1%|          | 5/511 [00:18<40:48,  4.84s/it]
Combination: ('avg_word_length',), F1 Score: 0.5451999272319883
  1%|          | 6/511 [00:24<42:23,  5.04s/it]
Combination: ('lexical_density',), F1 Score: 0.5130422256811207
  1%|▏         | 7/511 [00:26<33:29,  3.99s/it]
Combination: ('flesch_reading_ease',), F1 Score: 0.5334298326568333
  2%|▏         | 8/511 [00:27<27:01,  3.22s/it]
Combination: ('gunning_fog_index',), F1 Score: 0.5149184339056413
  2%|▏         | 9/511 [00:28<19:27,  2.33s/it]
Combination: ('grammatical_errors',), F1 Score: 0.5673935329265959
  2%|▏         | 10/511 [00:29<18:14,  2.19s/it]
Combination: ('word_count', 'sentence_count'), F1 Score: 0.5781793307872086
  2%|▏         | 11/511 [00:33<21:04,  2.53s/it]
Combination: ('word_count', 'lexical_diversity'), F1 Score: 0.3285401045974136
  2%|▏         | 12/511 [00:36<21:45,  2.62s/it]
Combination: ('word_count', 'avg_sentence_length'), F1 Score: 0.581197587698105
  3%|▎         | 13/511 [00:40<25:43,  3.10s/it]
Combination: ('word_count', 'avg_word_length'), F1 Score: 0.6354345419508043
  3%|▎         | 14/511 [00:43<27:11,  3.28s/it]
Combination: ('word_count', 'lexical_density'), F1 Score: 0.5621922770717676
  3%|▎         | 15/511 [00:47<26:46,  3.24s/it]
Combination: ('word_count', 'flesch_reading_ease'), F1 Score: 0.6347543076515039
  3%|▎         | 16/511 [00:50<26:22,  3.20s/it]
Combination: ('word_count', 'gunning_fog_index'), F1 Score: 0.6142580510773772
  3%|▎         | 17/511 [00:52<23:27,  2.85s/it]
Combination: ('word_count', 'grammatical_errors'), F1 Score: 0.5884307170863903
  4%|▎         | 18/511 [00:55<24:10,  2.94s/it]
Combination: ('sentence_count', 'lexical_diversity'), F1 Score: 0.3131269550210334
  4%|▎         | 19/511 [00:57<22:13,  2.71s/it]
Combination: ('sentence_count', 'avg_sentence_length'), F1 Score: 0.5751728110599079
  4%|▍         | 20/511 [01:01<24:48,  3.03s/it]
Combination: ('sentence_count', 'avg_word_length'), F1 Score: 0.5467811532706065
  4%|▍         | 21/511 [01:04<25:46,  3.16s/it]
Combination: ('sentence_count', 'lexical_density'), F1 Score: 0.5143392828172517
  4%|▍         | 22/511 [01:07<24:26,  3.00s/it]
Combination: ('sentence_count', 'flesch_reading_ease'), F1 Score: 0.5324658703088756
  5%|▍         | 23/511 [01:10<23:33,  2.90s/it]
Combination: ('sentence_count', 'gunning_fog_index'), F1 Score: 0.557775783801921
  5%|▍         | 24/511 [01:10<18:23,  2.27s/it]
Combination: ('sentence_count', 'grammatical_errors'), F1 Score: 0.5068151243972006
  5%|▍         | 25/511 [01:15<25:12,  3.11s/it]
Combination: ('lexical_diversity', 'avg_sentence_length'), F1 Score: 0.4185710636929949
  5%|▌         | 26/511 [01:21<30:47,  3.81s/it]
Combination: ('lexical_diversity', 'avg_word_length'), F1 Score: 0.399046560204838
  5%|▌         | 27/511 [01:26<34:30,  4.28s/it]
Combination: ('lexical_diversity', 'lexical_density'), F1 Score: 0.42841556631556105
  5%|▌         | 28/511 [01:31<34:34,  4.30s/it]
Combination: ('lexical_diversity', 'flesch_reading_ease'), F1 Score: 0.39159630268722945
  6%|▌         | 29/511 [01:35<34:47,  4.33s/it]
Combination: ('lexical_diversity', 'gunning_fog_index'), F1 Score: 0.4019524413124519
  6%|▌         | 30/511 [01:38<32:12,  4.02s/it]
Combination: ('lexical_diversity', 'grammatical_errors'), F1 Score: 0.5304392740591177
  6%|▌         | 31/511 [01:43<34:51,  4.36s/it]
Combination: ('avg_sentence_length', 'avg_word_length'), F1 Score: 0.5806375198466547
  6%|▋         | 32/511 [01:49<37:08,  4.65s/it]
Combination: ('avg_sentence_length', 'lexical_density'), F1 Score: 0.5325031630615753
  6%|▋         | 33/511 [01:52<33:57,  4.26s/it]
Combination: ('avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.5394781687145528
  7%|▋         | 34/511 [01:56<32:23,  4.08s/it]
Combination: ('avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.557700482608587
  7%|▋         | 35/511 [01:59<30:25,  3.83s/it]
Combination: ('avg_sentence_length', 'grammatical_errors'), F1 Score: 0.5572676415781916
  7%|▋         | 36/511 [02:05<35:06,  4.44s/it]
Combination: ('avg_word_length', 'lexical_density'), F1 Score: 0.5048075306121922
  7%|▋         | 37/511 [02:09<35:25,  4.48s/it]
Combination: ('avg_word_length', 'flesch_reading_ease'), F1 Score: 0.5719298649890798
  7%|▋         | 38/511 [02:14<35:44,  4.53s/it]
Combination: ('avg_word_length', 'gunning_fog_index'), F1 Score: 0.5587425440409193
  8%|▊         | 39/511 [02:18<33:46,  4.29s/it]
Combination: ('avg_word_length', 'grammatical_errors'), F1 Score: 0.5603443868728819
  8%|▊         | 40/511 [02:22<34:07,  4.35s/it]
Combination: ('lexical_density', 'flesch_reading_ease'), F1 Score: 0.5065560086678846
  8%|▊         | 41/511 [02:27<34:26,  4.40s/it]
Combination: ('lexical_density', 'gunning_fog_index'), F1 Score: 0.5104455202476003
  8%|▊         | 42/511 [02:30<32:32,  4.16s/it]
Combination: ('lexical_density', 'grammatical_errors'), F1 Score: 0.531801327105512
  8%|▊         | 43/511 [02:34<30:03,  3.85s/it]
Combination: ('flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5195677546073252
  9%|▊         | 44/511 [02:36<27:13,  3.50s/it]
Combination: ('flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5599168930180876
  9%|▉         | 45/511 [02:39<25:23,  3.27s/it]
Combination: ('gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5463021009146292
  9%|▉         | 46/511 [02:42<24:15,  3.13s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity'), F1 Score: 0.3279685975109468
  9%|▉         | 47/511 [02:44<22:15,  2.88s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length'), F1 Score: 0.5731884590159317
  9%|▉         | 48/511 [02:47<23:07,  3.00s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length'), F1 Score: 0.6568308725272163
 10%|▉         | 49/511 [02:50<23:17,  3.03s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density'), F1 Score: 0.5915989544933236
 10%|▉         | 50/511 [02:53<22:17,  2.90s/it]
Combination: ('word_count', 'sentence_count', 'flesch_reading_ease'), F1 Score: 0.6426512857590816
 10%|▉         | 51/511 [02:56<21:42,  2.83s/it]
Combination: ('word_count', 'sentence_count', 'gunning_fog_index'), F1 Score: 0.636703036573484
 10%|█         | 52/511 [02:58<19:44,  2.58s/it]
Combination: ('word_count', 'sentence_count', 'grammatical_errors'), F1 Score: 0.6072229900376369
 10%|█         | 53/511 [03:01<21:50,  2.86s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length'), F1 Score: 0.3527367527868627
 11%|█         | 54/511 [03:05<23:49,  3.13s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length'), F1 Score: 0.3786238926474296
 11%|█         | 55/511 [03:09<25:03,  3.30s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density'), F1 Score: 0.41097863175785254
 11%|█         | 56/511 [03:12<25:11,  3.32s/it]
Combination: ('word_count', 'lexical_diversity', 'flesch_reading_ease'), F1 Score: 0.39322472338102915
 11%|█         | 57/511 [03:15<25:17,  3.34s/it]
Combination: ('word_count', 'lexical_diversity', 'gunning_fog_index'), F1 Score: 0.34389923646802756
 11%|█▏        | 58/511 [03:18<23:46,  3.15s/it]
Combination: ('word_count', 'lexical_diversity', 'grammatical_errors'), F1 Score: 0.424053661341797
 12%|█▏        | 59/511 [03:22<25:48,  3.43s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.6684058051757431
 12%|█▏        | 60/511 [03:26<26:55,  3.58s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.6053973324859676
 12%|█▏        | 61/511 [03:29<26:11,  3.49s/it]
Combination: ('word_count', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.6606045081967213
 12%|█▏        | 62/511 [03:33<25:57,  3.47s/it]
Combination: ('word_count', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.6511417528948016
 12%|█▏        | 63/511 [03:36<24:35,  3.29s/it]
Combination: ('word_count', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.6122025218054366
 13%|█▎        | 64/511 [03:40<27:11,  3.65s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density'), F1 Score: 0.6191862907064654
 13%|█▎        | 65/511 [03:44<27:39,  3.72s/it]
Combination: ('word_count', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.6685754473950591
 13%|█▎        | 66/511 [03:48<27:57,  3.77s/it]
Combination: ('word_count', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.6646966542899047
 13%|█▎        | 67/511 [03:51<26:23,  3.57s/it]
Combination: ('word_count', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.6709420185425374
 13%|█▎        | 68/511 [03:55<27:02,  3.66s/it]
Combination: ('word_count', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6231024410772792
 14%|█▎        | 69/511 [03:59<27:28,  3.73s/it]
Combination: ('word_count', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6217199432451683
 14%|█▎        | 70/511 [04:02<25:52,  3.52s/it]
Combination: ('word_count', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5960915543026458
 14%|█▍        | 71/511 [04:05<25:06,  3.42s/it]
Combination: ('word_count', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6564442785530525
 14%|█▍        | 72/511 [04:08<23:17,  3.18s/it]
Combination: ('word_count', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6674084272540983
 14%|█▍        | 73/511 [04:10<22:10,  3.04s/it]
Combination: ('word_count', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6403099562324346
 14%|█▍        | 74/511 [04:14<22:17,  3.06s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length'), F1 Score: 0.33220907582425513
 15%|█▍        | 75/511 [04:17<23:24,  3.22s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length'), F1 Score: 0.32686153667127654
 15%|█▍        | 76/511 [04:21<24:01,  3.31s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density'), F1 Score: 0.3279866386203296
 15%|█▌        | 77/511 [04:24<23:38,  3.27s/it]
Combination: ('sentence_count', 'lexical_diversity', 'flesch_reading_ease'), F1 Score: 0.32718991485817667
 15%|█▌        | 78/511 [04:27<23:24,  3.24s/it]
Combination: ('sentence_count', 'lexical_diversity', 'gunning_fog_index'), F1 Score: 0.3249312140437047
 15%|█▌        | 79/511 [04:29<21:35,  3.00s/it]
Combination: ('sentence_count', 'lexical_diversity', 'grammatical_errors'), F1 Score: 0.3660024202930647
 16%|█▌        | 80/511 [04:33<22:32,  3.14s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.6579930310626778
 16%|█▌        | 81/511 [04:36<23:04,  3.22s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.6021732501293386
 16%|█▌        | 82/511 [04:39<22:10,  3.10s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.6511150953696092
 16%|█▌        | 83/511 [04:42<21:40,  3.04s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.6385157701851092
 16%|█▋        | 84/511 [04:44<19:59,  2.81s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.605821327417145
 17%|█▋        | 85/511 [04:48<22:23,  3.15s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density'), F1 Score: 0.5115427501153769
 17%|█▋        | 86/511 [04:52<22:50,  3.22s/it]
Combination: ('sentence_count', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.6062608368301692
 17%|█▋        | 87/511 [04:55<23:09,  3.28s/it]
Combination: ('sentence_count', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.6131999381119901
 17%|█▋        | 88/511 [04:58<21:34,  3.06s/it]
Combination: ('sentence_count', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.5727918661842195
 17%|█▋        | 89/511 [05:01<22:08,  3.15s/it]
Combination: ('sentence_count', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.5488401418600728
 18%|█▊        | 90/511 [05:04<22:30,  3.21s/it]
Combination: ('sentence_count', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.5757991685663704
 18%|█▊        | 91/511 [05:07<21:05,  3.01s/it]
Combination: ('sentence_count', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5407394706869313
 18%|█▊        | 92/511 [05:10<20:33,  2.94s/it]
Combination: ('sentence_count', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6003260763110745
 18%|█▊        | 93/511 [05:12<18:50,  2.71s/it]
Combination: ('sentence_count', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5604362235814824
 18%|█▊        | 94/511 [05:14<17:46,  2.56s/it]
Combination: ('sentence_count', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5691216782991682
 19%|█▊        | 95/511 [05:19<22:19,  3.22s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.40016000342963265
 19%|█▉        | 96/511 [05:24<25:52,  3.74s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.43099411988279435
 19%|█▉        | 97/511 [05:28<26:12,  3.80s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.43198539327607266
 19%|█▉        | 98/511 [05:32<26:29,  3.85s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.43433209047166466
 19%|█▉        | 99/511 [05:35<25:37,  3.73s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.5150863691181217
 20%|█▉        | 100/511 [05:40<28:15,  4.12s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density'), F1 Score: 0.39162992454154605
 20%|█▉        | 101/511 [05:45<28:57,  4.24s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.3911173253631275
 20%|█▉        | 102/511 [05:49<29:38,  4.35s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.3909389407781691
 20%|██        | 103/511 [05:53<27:34,  4.06s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.42107680965130667
 20%|██        | 104/511 [05:57<28:27,  4.20s/it]
Combination: ('lexical_diversity', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.42380893253866536
 21%|██        | 105/511 [06:02<29:16,  4.33s/it]
Combination: ('lexical_diversity', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.42899388971721686
 21%|██        | 106/511 [06:05<27:19,  4.05s/it]
Combination: ('lexical_diversity', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5263337145028262
 21%|██        | 107/511 [06:09<27:00,  4.01s/it]
Combination: ('lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.41531866769550435
 21%|██        | 108/511 [06:12<24:48,  3.69s/it]
Combination: ('lexical_diversity', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4436629076025781
 21%|██▏       | 109/511 [06:15<23:26,  3.50s/it]
Combination: ('lexical_diversity', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4370421174901694
 22%|██▏       | 110/511 [06:20<26:22,  3.95s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.5557410602269861
 22%|██▏       | 111/511 [06:24<26:44,  4.01s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.5584862089927465
 22%|██▏       | 112/511 [06:29<27:18,  4.11s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.5881902512095633
 22%|██▏       | 113/511 [06:32<26:10,  3.95s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.6215546262909153
 22%|██▏       | 114/511 [06:36<26:16,  3.97s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.5259449941003794
 23%|██▎       | 115/511 [06:40<26:38,  4.04s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.5441917443832119
 23%|██▎       | 116/511 [06:44<25:41,  3.90s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5646554314560304
 23%|██▎       | 117/511 [06:47<24:37,  3.75s/it]
Combination: ('avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5456749038695368
 23%|██▎       | 118/511 [06:50<22:59,  3.51s/it]
Combination: ('avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5731842678381166
 23%|██▎       | 119/511 [06:53<22:00,  3.37s/it]
Combination: ('avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5702348446647137
 23%|██▎       | 120/511 [06:58<24:37,  3.78s/it]
Combination: ('avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.5349149784525442
 24%|██▎       | 121/511 [07:03<26:26,  4.07s/it]
Combination: ('avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.5119782013406798
 24%|██▍       | 122/511 [07:07<25:41,  3.96s/it]
Combination: ('avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5529901785904859
 24%|██▍       | 123/511 [07:11<25:52,  4.00s/it]
Combination: ('avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5462112982418671
 24%|██▍       | 124/511 [07:14<24:19,  3.77s/it]
Combination: ('avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6027833338028565
 24%|██▍       | 125/511 [07:17<23:16,  3.62s/it]
Combination: ('avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6123227321563494
 25%|██▍       | 126/511 [07:21<23:56,  3.73s/it]
Combination: ('lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.47505815482433733
 25%|██▍       | 127/511 [07:24<22:59,  3.59s/it]
Combination: ('lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5399490568224459
 25%|██▌       | 128/511 [07:28<22:24,  3.51s/it]
Combination: ('lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5577845284167662
 25%|██▌       | 129/511 [07:30<20:52,  3.28s/it]
Combination: ('flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5661780809108155
 25%|██▌       | 130/511 [07:35<24:06,  3.80s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length'), F1 Score: 0.35679255549478095
 26%|██▌       | 131/511 [07:41<26:45,  4.22s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length'), F1 Score: 0.4232381533821685
 26%|██▌       | 132/511 [07:46<28:32,  4.52s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density'), F1 Score: 0.37593598458312255
 26%|██▌       | 133/511 [07:51<28:42,  4.56s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'flesch_reading_ease'), F1 Score: 0.424973185419415
 26%|██▌       | 134/511 [07:55<28:47,  4.58s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'gunning_fog_index'), F1 Score: 0.3738502595282717
 26%|██▋       | 135/511 [07:59<27:17,  4.35s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'grammatical_errors'), F1 Score: 0.40182560488419505
 27%|██▋       | 136/511 [08:05<29:29,  4.72s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.6543070224167491
 27%|██▋       | 137/511 [08:10<30:41,  4.92s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.594778993343015
 27%|██▋       | 138/511 [08:15<30:02,  4.83s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.6491272430251137
 27%|██▋       | 139/511 [08:19<29:49,  4.81s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.6386253122687728
 27%|██▋       | 140/511 [08:23<27:52,  4.51s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.605202137312229
 28%|██▊       | 141/511 [08:29<30:32,  4.95s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density'), F1 Score: 0.6423173610030861
 28%|██▊       | 142/511 [08:35<31:17,  5.09s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.6631003316021011
 28%|██▊       | 143/511 [08:40<32:01,  5.22s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.6736151225500173
 28%|██▊       | 144/511 [08:44<30:08,  4.93s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.6722442642746249
 28%|██▊       | 145/511 [08:50<30:35,  5.01s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6403999424639908
 29%|██▊       | 146/511 [08:55<30:58,  5.09s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6239864635126864
 29%|██▉       | 147/511 [08:59<28:58,  4.78s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5960306677018634
 29%|██▉       | 148/511 [09:03<28:17,  4.68s/it]
Combination: ('word_count', 'sentence_count', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6618701040191599
 29%|██▉       | 149/511 [09:07<26:06,  4.33s/it]
Combination: ('word_count', 'sentence_count', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.666241124099894
 29%|██▉       | 150/511 [09:10<24:41,  4.10s/it]
Combination: ('word_count', 'sentence_count', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6377899140564764
 30%|██▉       | 151/511 [09:17<28:35,  4.77s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.42230715253474316
 30%|██▉       | 152/511 [09:23<31:16,  5.23s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.37072042214362366
 30%|██▉       | 153/511 [09:29<32:01,  5.37s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.4085395952330583
 30%|███       | 154/511 [09:34<32:32,  5.47s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.37219695397528135
 30%|███       | 155/511 [09:39<31:29,  5.31s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.40284906299814477
 31%|███       | 156/511 [09:46<33:55,  5.73s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density'), F1 Score: 0.38940087430369497
 31%|███       | 157/511 [09:52<34:37,  5.87s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.4074337338479208
 31%|███       | 158/511 [09:58<35:10,  5.98s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.4076217163144067
 31%|███       | 159/511 [10:03<33:20,  5.68s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.48610678025790865
 31%|███▏      | 160/511 [10:10<33:59,  5.81s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4041865476440821
 32%|███▏      | 161/511 [10:16<34:34,  5.93s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.3683195675093155
 32%|███▏      | 162/511 [10:21<32:49,  5.64s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'grammatical_errors'), F1 Score: 0.42930040134615827
 32%|███▏      | 163/511 [10:27<33:01,  5.69s/it]
Combination: ('word_count', 'lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.3923294312430807
 32%|███▏      | 164/511 [10:31<31:00,  5.36s/it]
Combination: ('word_count', 'lexical_diversity', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4371744676226232
 32%|███▏      | 165/511 [10:36<29:46,  5.16s/it]
Combination: ('word_count', 'lexical_diversity', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.40608752873177884
 32%|███▏      | 166/511 [10:43<33:14,  5.78s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.6402129683094461
 33%|███▎      | 167/511 [10:50<34:16,  5.98s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.6729632541512365
 33%|███▎      | 168/511 [10:56<35:08,  6.15s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.6833324991555201
 33%|███▎      | 169/511 [11:01<33:30,  5.88s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.685870201085971
 33%|███▎      | 170/511 [11:07<33:53,  5.96s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6405803181782235
 33%|███▎      | 171/511 [11:14<34:36,  6.11s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6329555876261028
 34%|███▎      | 172/511 [11:19<33:08,  5.87s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.6000724931885765
 34%|███▍      | 173/511 [11:25<32:30,  5.77s/it]
Combination: ('word_count', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6683490053430615
 34%|███▍      | 174/511 [11:29<30:29,  5.43s/it]
Combination: ('word_count', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.669129988226158
 34%|███▍      | 175/511 [11:34<29:21,  5.24s/it]
Combination: ('word_count', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6384898728387626
 34%|███▍      | 176/511 [11:41<32:11,  5.76s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6601833489841002
 35%|███▍      | 177/511 [11:48<34:06,  6.13s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6393958314158111
 35%|███▍      | 178/511 [11:54<33:05,  5.96s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.6322107083573443
 35%|███▌      | 179/511 [12:00<33:30,  6.06s/it]
Combination: ('word_count', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6627432734506378
 35%|███▌      | 180/511 [12:05<31:46,  5.76s/it]
Combination: ('word_count', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6770483056048037
 35%|███▌      | 181/511 [12:10<30:33,  5.56s/it]
Combination: ('word_count', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6713285110011158
 36%|███▌      | 182/511 [12:16<31:24,  5.73s/it]
Combination: ('word_count', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.633971654764945
 36%|███▌      | 183/511 [12:21<30:01,  5.49s/it]
Combination: ('word_count', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6343357377531372
 36%|███▌      | 184/511 [12:26<29:19,  5.38s/it]
Combination: ('word_count', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6238458450421518
 36%|███▌      | 185/511 [12:31<27:38,  5.09s/it]
Combination: ('word_count', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6619774818826951
 36%|███▋      | 186/511 [12:36<28:17,  5.22s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.37345665648685483
 37%|███▋      | 187/511 [12:42<28:36,  5.30s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.34971825954899793
 37%|███▋      | 188/511 [12:47<27:54,  5.18s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.3521298910800049
 37%|███▋      | 189/511 [12:52<27:19,  5.09s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.36880549967545023
 37%|███▋      | 190/511 [12:56<25:41,  4.80s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.39618841285338713
 37%|███▋      | 191/511 [13:02<28:12,  5.29s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density'), F1 Score: 0.3373696984842805
 38%|███▊      | 192/511 [13:08<28:52,  5.43s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.34343324994731356
 38%|███▊      | 193/511 [13:14<29:21,  5.54s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.3520621719198622
 38%|███▊      | 194/511 [13:18<27:34,  5.22s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.377044245845905
 38%|███▊      | 195/511 [13:24<28:30,  5.41s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.35040521198220936
 38%|███▊      | 196/511 [13:30<29:10,  5.56s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.33930367814916373
 39%|███▊      | 197/511 [13:34<27:22,  5.23s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'grammatical_errors'), F1 Score: 0.3701860796417286
 39%|███▊      | 198/511 [13:40<27:26,  5.26s/it]
Combination: ('sentence_count', 'lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.3425423541136086
 39%|███▉      | 199/511 [13:44<25:26,  4.89s/it]
Combination: ('sentence_count', 'lexical_diversity', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.39580298861735985
 39%|███▉      | 200/511 [13:48<24:06,  4.65s/it]
Combination: ('sentence_count', 'lexical_diversity', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.39292768199651074
 39%|███▉      | 201/511 [13:54<26:29,  5.13s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.6079848593972362
 40%|███▉      | 202/511 [14:00<27:00,  5.24s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.6687523003312477
 40%|███▉      | 203/511 [14:05<27:40,  5.39s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.6571758103251766
 40%|███▉      | 204/511 [14:10<26:06,  5.10s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.6347450819333375
 40%|████      | 205/511 [14:15<26:27,  5.19s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6308245530605965
 40%|████      | 206/511 [14:21<26:57,  5.30s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6232859460530271
 41%|████      | 207/511 [14:25<25:36,  5.05s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.576496575022637
 41%|████      | 208/511 [14:30<24:59,  4.95s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6533549500093032
 41%|████      | 209/511 [14:34<23:15,  4.62s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6329740027822991
 41%|████      | 210/511 [14:38<22:15,  4.44s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6287764148530663
 41%|████▏     | 211/511 [14:44<24:59,  5.00s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.5765828683541329
 41%|████▏     | 212/511 [14:50<26:51,  5.39s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.5776755357745318
 42%|████▏     | 213/511 [14:55<26:02,  5.24s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5343313343744931
 42%|████▏     | 214/511 [15:01<26:29,  5.35s/it]
Combination: ('sentence_count', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6089785876674606
 42%|████▏     | 215/511 [15:05<25:03,  5.08s/it]
Combination: ('sentence_count', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6046046702994423
 42%|████▏     | 216/511 [15:10<23:55,  4.87s/it]
Combination: ('sentence_count', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.622113415070504
 42%|████▏     | 217/511 [15:15<24:40,  5.03s/it]
Combination: ('sentence_count', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5500816538556719
 43%|████▎     | 218/511 [15:19<23:21,  4.78s/it]
Combination: ('sentence_count', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5307337672491375
 43%|████▎     | 219/511 [15:24<22:36,  4.64s/it]
Combination: ('sentence_count', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.560846658352247
 43%|████▎     | 220/511 [15:27<21:11,  4.37s/it]
Combination: ('sentence_count', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5919178643431136
 43%|████▎     | 221/511 [15:35<26:36,  5.50s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.40113444799791204
 43%|████▎     | 222/511 [15:43<29:02,  6.03s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.4039351731347075
 44%|████▎     | 223/511 [15:50<30:58,  6.45s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.41048589196492274
 44%|████▍     | 224/511 [15:56<29:56,  6.26s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.43848349436061573
 44%|████▍     | 225/511 [16:03<30:54,  6.48s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4583195256876611
 44%|████▍     | 226/511 [16:10<31:46,  6.69s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.46169337616647843
 44%|████▍     | 227/511 [16:16<30:35,  6.46s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5005981408640579
 45%|████▍     | 228/511 [16:22<30:18,  6.42s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4494218717320704
 45%|████▍     | 229/511 [16:28<28:25,  6.05s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4508481286756786
 45%|████▌     | 230/511 [16:33<27:17,  5.83s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44503929848712037
 45%|████▌     | 231/511 [16:41<30:06,  6.45s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.40205157780147216
 45%|████▌     | 232/511 [16:49<32:20,  6.96s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.40191223567605505
 46%|████▌     | 233/511 [16:55<30:48,  6.65s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.45981052388444665
 46%|████▌     | 234/511 [17:02<31:36,  6.85s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.40685222754532135
 46%|████▌     | 235/511 [17:08<29:31,  6.42s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4420180447036134
 46%|████▌     | 236/511 [17:13<28:12,  6.15s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4203088408126461
 46%|████▋     | 237/511 [17:20<29:34,  6.48s/it]
Combination: ('lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.43947478850357985
 47%|████▋     | 238/511 [17:26<28:01,  6.16s/it]
Combination: ('lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4433695820206792
 47%|████▋     | 239/511 [17:31<27:09,  5.99s/it]
Combination: ('lexical_diversity', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.41534099581949796
 47%|████▋     | 240/511 [17:36<25:42,  5.69s/it]
Combination: ('lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44832031009441997
 47%|████▋     | 241/511 [17:44<27:53,  6.20s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.5344963365977277
 47%|████▋     | 242/511 [17:52<29:50,  6.65s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.545641311805785
 48%|████▊     | 243/511 [17:58<29:12,  6.54s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5832823562235121
 48%|████▊     | 244/511 [18:04<29:20,  6.59s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5619760297289231
 48%|████▊     | 245/511 [18:10<27:56,  6.30s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6174133797339965
 48%|████▊     | 246/511 [18:16<27:06,  6.14s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6201961254043924
 48%|████▊     | 247/511 [18:22<27:17,  6.20s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5039653474016736
 49%|████▊     | 248/511 [18:28<26:18,  6.00s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5457726172240669
 49%|████▊     | 249/511 [18:34<25:54,  5.93s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5704825931397577
 49%|████▉     | 250/511 [18:38<24:28,  5.63s/it]
Combination: ('avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5554642115324668
'''

In [8]:
import re
pattern = r"Combination: (\(.*?\)), F1 Score: ([0-9.]+)"

# Find all matches
matches = re.findall(pattern, result)

# Convert F1 scores to floats and find the max
max_combination = max(matches, key=lambda x: float(x[1]))

print("Combination with the highest F1 score:")
print("Combination:", max_combination[0], "F1 Score:", max_combination[1])


Combination with the highest F1 score:
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors') F1 Score: 0.685870201085971


In [10]:
print(len(feature_combinations))
feature_combinations=feature_combinations[250:]
print(len(feature_combinations))

# Iterate over combinations, train models, and evaluate
best_f1 = 0
best_metrics = None
best_combination = None

for combo in tqdm(feature_combinations):
    # Select features for this combination
    X_train_combo = X_train[list(combo)]
    X_test_combo = X_test[list(combo)]

    # Train the Random Forest model
    rf = RandomForestClassifier()
    rf.fit(X_train_combo, y_train)

    # Predict on the test set
    predicted_labels = rf.predict(X_test_combo)

    # Evaluate the model
    metrics = getMetrics(predicted_labels, y_test)
    
    print(f"Combination: {combo}, F1 Score: {metrics['f1']}")

    # Check if this is the best model so far
    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        best_metrics = metrics
        best_combination = combo

# Return the best F1 score, the corresponding combination of features, and the full metrics
best_f1, best_combination, best_metrics

511
261


  0%|          | 1/261 [00:07<32:53,  7.59s/it]

Combination: ('avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5090982892685956


  1%|          | 2/261 [00:13<28:38,  6.64s/it]

Combination: ('avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.573234068754731


...

100%|█████████▉| 260/261 [23:26<00:05,  5.28s/it]

Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4430971520730894


100%|██████████| 261/261 [23:33<00:00,  5.42s/it]

Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5011291430744145


(0.6874337451825074,
 ('word_count',
  'avg_sentence_length',
  'avg_word_length',
  'gunning_fog_index',
  'grammatical_errors'),
 {'f1': 0.6874337451825074,
  'confusion_matrix': [[1860, 640], [918, 1582]],
  'accuracy': 0.6884,
  'precision': 0.711971197119712,
  'recall': 0.6328,
  'auc': 0.6884})

In [12]:
result='''
  0%|          | 1/261 [00:07<32:53,  7.59s/it]
Combination: ('avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5090982892685956
  1%|          | 2/261 [00:13<28:38,  6.64s/it]
Combination: ('avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.573234068754731
  1%|          | 3/261 [00:19<27:16,  6.34s/it]
Combination: ('avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5811868057481065
  2%|▏         | 4/261 [00:24<25:25,  5.94s/it]
Combination: ('avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5964034054512016
  2%|▏         | 5/261 [00:30<24:18,  5.70s/it]
Combination: ('lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5297607361156172
  2%|▏         | 6/261 [00:35<23:54,  5.63s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length'), F1 Score: 0.446043914220129
  3%|▎         | 7/261 [00:41<23:39,  5.59s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density'), F1 Score: 0.3670433490026669
  3%|▎         | 8/261 [00:46<22:47,  5.40s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease'), F1 Score: 0.44064642887867045
  3%|▎         | 9/261 [00:51<22:08,  5.27s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'gunning_fog_index'), F1 Score: 0.3847769017597439
  4%|▍         | 10/261 [00:55<20:54,  5.00s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'grammatical_errors'), F1 Score: 0.408117446756187
  4%|▍         | 11/261 [01:01<21:45,  5.22s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density'), F1 Score: 0.4022688120352183
  5%|▍         | 12/261 [01:06<21:54,  5.28s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.42779726526737333
  5%|▍         | 13/261 [01:12<22:01,  5.33s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.41726270221514394
  5%|▌         | 14/261 [01:16<20:52,  5.07s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.4611436230237924
  6%|▌         | 15/261 [01:21<21:07,  5.15s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.40162637728622574
  6%|▌         | 16/261 [01:27<21:20,  5.22s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.3839987515708032
  7%|▋         | 17/261 [01:31<20:21,  5.01s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'grammatical_errors'), F1 Score: 0.41450926973027313
  7%|▋         | 18/261 [01:36<20:18,  5.01s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.42595396682053444
  7%|▋         | 19/261 [01:40<19:02,  4.72s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.44933957282301484
  8%|▊         | 20/261 [01:45<18:16,  4.55s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.41722716277304056
  8%|▊         | 21/261 [01:51<20:03,  5.02s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.6365764799982581
  8%|▊         | 22/261 [01:56<20:36,  5.17s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.6691235543059869
  9%|▉         | 23/261 [02:02<21:03,  5.31s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.6737143695991576
  9%|▉         | 24/261 [02:06<20:10,  5.11s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.668275010779054
 10%|▉         | 25/261 [02:12<20:18,  5.16s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6393681345683705
 10%|▉         | 26/261 [02:17<20:36,  5.26s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6269998657199516
 10%|█         | 27/261 [02:22<19:49,  5.08s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5988228245985311
 11%|█         | 28/261 [02:27<19:33,  5.04s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6589556778798973
 11%|█         | 29/261 [02:31<18:26,  4.77s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6645328925498855
 11%|█▏        | 30/261 [02:35<17:46,  4.62s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6331827172731486
 12%|█▏        | 31/261 [02:41<19:17,  5.03s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6529089094141867
 12%|█▏        | 32/261 [02:47<20:21,  5.33s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6447850381136648
 13%|█▎        | 33/261 [02:52<19:44,  5.20s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.6362194014488884
 13%|█▎        | 34/261 [02:58<19:59,  5.29s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6769348571445947
 13%|█▎        | 35/261 [03:02<18:59,  5.04s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6698192604682047
 14%|█▍        | 36/261 [03:07<18:24,  4.91s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.683643020135911
 14%|█▍        | 37/261 [03:12<18:49,  5.04s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6441435042227304
 15%|█▍        | 38/261 [03:16<17:59,  4.84s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6364610099999823
 15%|█▍        | 39/261 [03:21<17:31,  4.74s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6251249315410551
 15%|█▌        | 40/261 [03:25<16:36,  4.51s/it]
Combination: ('word_count', 'sentence_count', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.659195325822724
 16%|█▌        | 41/261 [03:32<18:50,  5.14s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.3977259008659971
 16%|█▌        | 42/261 [03:38<19:53,  5.45s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.43036184617560236
 16%|█▋        | 43/261 [03:44<20:36,  5.67s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.41267812596900894
 17%|█▋        | 44/261 [03:49<20:07,  5.56s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.4787965982761892
 17%|█▋        | 45/261 [03:55<20:41,  5.75s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.40975893648952855
 18%|█▊        | 46/261 [04:01<20:56,  5.85s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.39112561537234236
 18%|█▊        | 47/261 [04:07<20:19,  5.70s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.4046464929466348
 18%|█▊        | 48/261 [04:12<20:10,  5.68s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4146946948323956
 19%|█▉        | 49/261 [04:17<19:10,  5.43s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.44234764496065343
 19%|█▉        | 50/261 [04:22<18:32,  5.27s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.40087690497015405
 20%|█▉        | 51/261 [04:29<19:43,  5.63s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.41229797494610604
 20%|█▉        | 52/261 [04:35<20:35,  5.91s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.3986558314573527
 20%|██        | 53/261 [04:41<20:02,  5.78s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.46463744380728556
 21%|██        | 54/261 [04:47<20:17,  5.88s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.41914242345935343
 21%|██        | 55/261 [04:52<19:24,  5.65s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4312647289866457
 21%|██▏       | 56/261 [04:57<18:46,  5.49s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4792292381958042
 22%|██▏       | 57/261 [05:03<19:17,  5.68s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.38530163062712897
 22%|██▏       | 58/261 [05:08<18:34,  5.49s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4282877537816457
 23%|██▎       | 59/261 [05:13<18:06,  5.38s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44526624773726153
 23%|██▎       | 60/261 [05:18<17:22,  5.19s/it]
Combination: ('word_count', 'lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44730526191898795
 23%|██▎       | 61/261 [05:25<18:53,  5.67s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.660542631704758
 24%|██▍       | 62/261 [05:32<20:12,  6.09s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6593233477532444
 24%|██▍       | 63/261 [05:38<19:52,  6.02s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.6505116642489182
 25%|██▍       | 64/261 [05:44<20:08,  6.13s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6803627175073701
 25%|██▍       | 65/261 [05:50<19:18,  5.91s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.680101404846213
 25%|██▌       | 66/261 [05:55<18:47,  5.78s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6874337451825074
 26%|██▌       | 67/261 [06:01<19:05,  5.91s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6497635614009281
 26%|██▌       | 68/261 [06:07<18:21,  5.71s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6457223853266496
 26%|██▋       | 69/261 [06:12<18:01,  5.64s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6353514359836328
 27%|██▋       | 70/261 [06:17<17:12,  5.41s/it]
Combination: ('word_count', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6697938486485862
 27%|██▋       | 71/261 [06:24<18:22,  5.80s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6473968265714392
 28%|██▊       | 72/261 [06:29<18:05,  5.74s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.648579061152278
 28%|██▊       | 73/261 [06:35<17:57,  5.73s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6457308512084083
 28%|██▊       | 74/261 [06:40<17:20,  5.57s/it]
Combination: ('word_count', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.678033255516016
 29%|██▊       | 75/261 [06:45<16:47,  5.42s/it]
Combination: ('word_count', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6322789634084944
 29%|██▉       | 76/261 [06:51<17:15,  5.60s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.3741025568262436
 30%|██▉       | 77/261 [06:57<17:09,  5.60s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.3783775668081067
 30%|██▉       | 78/261 [07:02<17:08,  5.62s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.37489911916620544
 30%|███       | 79/261 [07:07<16:16,  5.37s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.42254994306433374
 31%|███       | 80/261 [07:13<16:19,  5.41s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.37924542177233755
 31%|███       | 81/261 [07:18<16:20,  5.45s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.380676085671221
 31%|███▏      | 82/261 [07:23<15:39,  5.25s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.37893498099760164
 32%|███▏      | 83/261 [07:28<15:25,  5.20s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.3923709310968294
 32%|███▏      | 84/261 [07:32<14:33,  4.94s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.41442203771284675
 33%|███▎      | 85/261 [07:37<14:00,  4.77s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.39731812723414717
 33%|███▎      | 86/261 [07:43<15:16,  5.24s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.34930799280565417
 33%|███▎      | 87/261 [07:49<16:07,  5.56s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.35965100123365196
 34%|███▎      | 88/261 [07:55<15:40,  5.44s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.3764668357401713
 34%|███▍      | 89/261 [08:00<15:51,  5.53s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.3443896665683402
 34%|███▍      | 90/261 [08:05<15:05,  5.30s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.3797081903459871
 35%|███▍      | 91/261 [08:10<14:31,  5.13s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.3969417330453839
 35%|███▌      | 92/261 [08:16<14:59,  5.32s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.36175137729260926
 36%|███▌      | 93/261 [08:20<14:24,  5.14s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.38057290529187787
 36%|███▌      | 94/261 [08:25<13:58,  5.02s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.3792253760118956
 36%|███▋      | 95/261 [08:29<13:19,  4.81s/it]
Combination: ('sentence_count', 'lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4040808543096873
 37%|███▋      | 96/261 [08:36<14:17,  5.20s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6287759724096207
 37%|███▋      | 97/261 [08:42<15:01,  5.50s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6285553212884336
 38%|███▊      | 98/261 [08:47<14:36,  5.38s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.5746650543781795
 38%|███▊      | 99/261 [08:52<14:44,  5.46s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6617928435365692
 38%|███▊      | 100/261 [08:57<14:01,  5.22s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6387292580484978
 39%|███▊      | 101/261 [09:02<13:32,  5.08s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6577709238072531
 39%|███▉      | 102/261 [09:07<13:47,  5.20s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6085789395267088
 39%|███▉      | 103/261 [09:12<13:13,  5.02s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5729476344151396
 40%|███▉      | 104/261 [09:17<12:53,  4.93s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6028042183971702
 40%|████      | 105/261 [09:21<12:14,  4.71s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6171541462731511
 41%|████      | 106/261 [09:27<13:13,  5.12s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5760526416097644
 41%|████      | 107/261 [09:32<13:05,  5.10s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5594471333266139
 41%|████▏     | 108/261 [09:37<12:59,  5.09s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5752596437185636
 42%|████▏     | 109/261 [09:42<12:35,  4.97s/it]
Combination: ('sentence_count', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6165965766962793
 42%|████▏     | 110/261 [09:46<12:09,  4.83s/it]
Combination: ('sentence_count', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5424620645654554
 43%|████▎     | 111/261 [09:54<14:05,  5.64s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4105945851435336
 43%|████▎     | 112/261 [10:01<15:24,  6.20s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.40580990893319124
 43%|████▎     | 113/261 [10:07<15:15,  6.19s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.4729505476269499
 44%|████▎     | 114/261 [10:14<15:39,  6.39s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.42908560934703455
 44%|████▍     | 115/261 [10:20<15:02,  6.18s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.44843721797618463
 44%|████▍     | 116/261 [10:26<14:39,  6.07s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4354756291005611
 45%|████▍     | 117/261 [10:33<15:01,  6.26s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.46360664441992205
 45%|████▌     | 118/261 [10:38<14:30,  6.09s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4672338729980423
 46%|████▌     | 119/261 [10:44<14:09,  5.99s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4621603948311168
 46%|████▌     | 120/261 [10:49<13:34,  5.78s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44155300906889455
 46%|████▋     | 121/261 [10:57<14:41,  6.29s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4374938346391679
 47%|████▋     | 122/261 [11:03<14:14,  6.15s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4423128751486961
 47%|████▋     | 123/261 [11:08<13:58,  6.08s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4349371703734473
 48%|████▊     | 124/261 [11:14<13:29,  5.91s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44612038499432033
 48%|████▊     | 125/261 [11:19<13:03,  5.76s/it]
Combination: ('lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.46053328892582474
 48%|████▊     | 126/261 [11:26<13:50,  6.15s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.5269547854041298
 49%|████▊     | 127/261 [11:33<13:44,  6.15s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5825825997338978
 49%|████▉     | 128/261 [11:39<13:41,  6.17s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5829554184837822
 49%|████▉     | 129/261 [11:45<13:16,  6.03s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5932203389830509
 50%|████▉     | 130/261 [11:50<12:53,  5.90s/it]
Combination: ('avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5330766260457698
 50%|█████     | 131/261 [11:56<12:47,  5.90s/it]
Combination: ('avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5664277509710959
 51%|█████     | 132/261 [12:02<12:37,  5.87s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density'), F1 Score: 0.4184557910181219
 51%|█████     | 133/261 [12:07<12:15,  5.74s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease'), F1 Score: 0.4693925324801998
 51%|█████▏    | 134/261 [12:13<11:59,  5.67s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index'), F1 Score: 0.43675751296778853
 52%|█████▏    | 135/261 [12:18<11:18,  5.39s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'grammatical_errors'), F1 Score: 0.47922803640383516
 52%|█████▏    | 136/261 [12:23<11:14,  5.40s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4299007148816645
 52%|█████▏    | 137/261 [12:28<11:09,  5.40s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.4196698915838849
 53%|█████▎    | 138/261 [12:33<10:41,  5.21s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.4334586615749386
 53%|█████▎    | 139/261 [12:38<10:31,  5.18s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.45393735016064735
 54%|█████▎    | 140/261 [12:43<09:59,  4.96s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.44959380022456574
 54%|█████▍    | 141/261 [12:47<09:36,  4.80s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4248848612300434
 54%|█████▍    | 142/261 [12:53<10:06,  5.10s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4367186015647569
 55%|█████▍    | 143/261 [12:59<10:27,  5.32s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.3957112059430051
 55%|█████▌    | 144/261 [13:04<10:09,  5.21s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.4540137676138667
 56%|█████▌    | 145/261 [13:09<10:15,  5.31s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4397108460260883
 56%|█████▌    | 146/261 [13:14<09:48,  5.12s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.47059546386000334
 56%|█████▋    | 147/261 [13:19<09:29,  5.00s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.47937844492635767
 57%|█████▋    | 148/261 [13:24<09:40,  5.14s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.40490119029084637
 57%|█████▋    | 149/261 [13:29<09:19,  4.99s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4148829297765897
 57%|█████▋    | 150/261 [13:33<09:05,  4.91s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.42686458076455325
 58%|█████▊    | 151/261 [13:38<08:42,  4.75s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4475285221720783
 58%|█████▊    | 152/261 [13:44<09:20,  5.14s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.6566338107644099
 59%|█████▊    | 153/261 [13:50<09:48,  5.45s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.6493980532786885
 59%|█████▉    | 154/261 [13:55<09:36,  5.39s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.6339383824352622
 59%|█████▉    | 155/261 [14:01<09:38,  5.45s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.672593072069405
 60%|█████▉    | 156/261 [14:06<09:11,  5.25s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.675301637558732
 60%|██████    | 157/261 [14:11<08:53,  5.13s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6862781180010966
 61%|██████    | 158/261 [14:16<09:01,  5.25s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6452926087902373
 61%|██████    | 159/261 [14:21<08:38,  5.09s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6382620051167285
 61%|██████▏   | 160/261 [14:26<08:26,  5.02s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6321296141543309
 62%|██████▏   | 161/261 [14:30<08:02,  4.82s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6654668559531589
 62%|██████▏   | 162/261 [14:36<08:30,  5.15s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6547407931979012
 62%|██████▏   | 163/261 [14:41<08:20,  5.11s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6489551438026436
 63%|██████▎   | 164/261 [14:46<08:13,  5.09s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6495103389206789
 63%|██████▎   | 165/261 [14:51<07:57,  4.97s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6758121867655621
 64%|██████▎   | 166/261 [14:55<07:41,  4.86s/it]
Combination: ('word_count', 'sentence_count', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6423222940614752
 64%|██████▍   | 167/261 [15:02<08:21,  5.34s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4132744737801243
 64%|██████▍   | 168/261 [15:08<08:48,  5.68s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.402947666169869
 65%|██████▍   | 169/261 [15:14<08:41,  5.67s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.46539840732811644
 65%|██████▌   | 170/261 [15:20<08:45,  5.78s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.44041263240480927
 66%|██████▌   | 171/261 [15:25<08:26,  5.63s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4776807608723329
 66%|██████▌   | 172/261 [15:31<08:15,  5.57s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.45141173926930017
 66%|██████▋   | 173/261 [15:37<08:23,  5.72s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4468414647638013
 67%|██████▋   | 174/261 [15:42<08:06,  5.59s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.43389966163156957
 67%|██████▋   | 175/261 [15:47<07:56,  5.54s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.40940872007347007
 67%|██████▋   | 176/261 [15:52<07:37,  5.38s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.45253203931815356
 68%|██████▊   | 177/261 [15:59<07:58,  5.70s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4251727279647961
 68%|██████▊   | 178/261 [16:04<07:46,  5.62s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4583471622967142
 69%|██████▊   | 179/261 [16:10<07:38,  5.59s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.46158171452525637
 69%|██████▉   | 180/261 [16:15<07:23,  5.48s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.48270878426297453
 69%|██████▉   | 181/261 [16:20<07:11,  5.40s/it]
Combination: ('word_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4519913403113398
 70%|██████▉   | 182/261 [16:27<07:36,  5.78s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6706885083013705
 70%|███████   | 183/261 [16:33<07:30,  5.78s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6532712517625041
 70%|███████   | 184/261 [16:39<07:26,  5.80s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6545737343032816
 71%|███████   | 185/261 [16:44<07:12,  5.69s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6810421568386728
 71%|███████▏  | 186/261 [16:49<06:58,  5.58s/it]
Combination: ('word_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6359030113361175
 72%|███████▏  | 187/261 [16:55<06:53,  5.58s/it]
Combination: ('word_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6434007098361856
 72%|███████▏  | 188/261 [17:01<06:53,  5.67s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.39041736091356116
 72%|███████▏  | 189/261 [17:07<06:52,  5.73s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.37643862816498197
 73%|███████▎  | 190/261 [17:12<06:34,  5.56s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.4224136508408007
 73%|███████▎  | 191/261 [17:17<06:28,  5.55s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.38043030460270444
 74%|███████▎  | 192/261 [17:22<06:07,  5.32s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4106796388578889
 74%|███████▍  | 193/261 [17:27<05:53,  5.20s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.433382440188517
 74%|███████▍  | 194/261 [17:32<05:54,  5.29s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.3914498775032311
 75%|███████▍  | 195/261 [17:37<05:40,  5.16s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4140671412519138
 75%|███████▌  | 196/261 [17:42<05:30,  5.08s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.41221142519920156
 75%|███████▌  | 197/261 [17:47<05:14,  4.92s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.42310234453647644
 76%|███████▌  | 198/261 [17:53<05:33,  5.29s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.3619552243201227
 76%|███████▌  | 199/261 [17:58<05:25,  5.25s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.39086464496935885
 77%|███████▋  | 200/261 [18:03<05:18,  5.22s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.39042878848507956
 77%|███████▋  | 201/261 [18:08<05:06,  5.11s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.3920428001868669
 77%|███████▋  | 202/261 [18:13<04:57,  5.04s/it]
Combination: ('sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.38897208817219214
 78%|███████▊  | 203/261 [18:19<05:09,  5.34s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.617914664932834
 78%|███████▊  | 204/261 [18:24<05:01,  5.29s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.5831199840415549
 79%|███████▊  | 205/261 [18:30<04:57,  5.31s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.591172282699683
 79%|███████▉  | 206/261 [18:34<04:45,  5.19s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6376448263704351
 79%|███████▉  | 207/261 [18:39<04:34,  5.08s/it]
Combination: ('sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5670182200695519
 80%|███████▉  | 208/261 [18:44<04:30,  5.10s/it]
Combination: ('sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5709199366066433
 80%|████████  | 209/261 [18:52<04:57,  5.71s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4049982508566735
 80%|████████  | 210/261 [18:58<04:56,  5.81s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.450853539127032
 81%|████████  | 211/261 [19:04<04:55,  5.92s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4498513611861501
 81%|████████  | 212/261 [19:10<04:48,  5.88s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4398848117386124
 82%|████████▏ | 213/261 [19:15<04:39,  5.83s/it]
Combination: ('lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44225679725602807
 82%|████████▏ | 214/261 [19:21<04:36,  5.88s/it]
Combination: ('lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.43633725951461133
 82%|████████▏ | 215/261 [19:27<04:33,  5.94s/it]
Combination: ('avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.5588645271322914
 83%|████████▎ | 216/261 [19:33<04:26,  5.91s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease'), F1 Score: 0.4557822294289738
 83%|████████▎ | 217/261 [19:39<04:19,  5.90s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index'), F1 Score: 0.43952496421098897
 84%|████████▎ | 218/261 [19:44<04:03,  5.67s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'grammatical_errors'), F1 Score: 0.4731132833926593
 84%|████████▍ | 219/261 [19:50<03:56,  5.63s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.47222222222222227
 84%|████████▍ | 220/261 [19:55<03:41,  5.40s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.4882404047487783
 85%|████████▍ | 221/261 [20:00<03:30,  5.26s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4773397986863939
 85%|████████▌ | 222/261 [20:05<03:27,  5.31s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.45541398349810813
 85%|████████▌ | 223/261 [20:10<03:15,  5.15s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.44305619480796865
 86%|████████▌ | 224/261 [20:15<03:07,  5.06s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4488510464349763
 86%|████████▌ | 225/261 [20:19<02:56,  4.91s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.47117518396583835
 87%|████████▋ | 226/261 [20:25<03:00,  5.16s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.41826860726566495
 87%|████████▋ | 227/261 [20:30<02:53,  5.12s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.47223224445616274
 87%|████████▋ | 228/261 [20:35<02:48,  5.11s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4661671747947871
 88%|████████▊ | 229/261 [20:40<02:40,  5.03s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4851544811463395
 88%|████████▊ | 230/261 [20:45<02:33,  4.95s/it]
Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.43116926680246137
 89%|████████▊ | 231/261 [20:51<02:37,  5.26s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.6621119030930533
 89%|████████▉ | 232/261 [20:56<02:31,  5.24s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.6533398129228145
 89%|████████▉ | 233/261 [21:01<02:26,  5.24s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6532005313960444
 90%|████████▉ | 234/261 [21:06<02:19,  5.16s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6783167349581788
 90%|█████████ | 235/261 [21:11<02:11,  5.07s/it]
Combination: ('word_count', 'sentence_count', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6373572240296541
 90%|█████████ | 236/261 [21:16<02:07,  5.08s/it]
Combination: ('word_count', 'sentence_count', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6509347467377321
 91%|█████████ | 237/261 [21:22<02:10,  5.45s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.44600169240533105
 91%|█████████ | 238/261 [21:28<02:06,  5.51s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.47118044031419404
 92%|█████████▏| 239/261 [21:34<02:02,  5.58s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.46165115628541353
 92%|█████████▏| 240/261 [21:39<01:56,  5.53s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4689765167840686
 92%|█████████▏| 241/261 [21:44<01:49,  5.49s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4374324443602793
 93%|█████████▎| 242/261 [21:50<01:44,  5.52s/it]
Combination: ('word_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.47027898404545687
 93%|█████████▎| 243/261 [21:56<01:41,  5.61s/it]
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.6511304925710568
 93%|█████████▎| 244/261 [22:02<01:36,  5.69s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.40177422317184114
 94%|█████████▍| 245/261 [22:07<01:28,  5.54s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'grammatical_errors'), F1 Score: 0.42577730581641765
 94%|█████████▍| 246/261 [22:12<01:21,  5.45s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.44416388954809927
 95%|█████████▍| 247/261 [22:17<01:14,  5.31s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.42986703244858726
 95%|█████████▌| 248/261 [22:22<01:07,  5.19s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4156504065040651
 95%|█████████▌| 249/261 [22:27<01:02,  5.19s/it]
Combination: ('sentence_count', 'lexical_diversity', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.3926320884762368
 96%|█████████▌| 250/261 [22:33<00:57,  5.22s/it]
'''

In [13]:
import re
pattern = r"Combination: (\(.*?\)), F1 Score: ([0-9.]+)"

# Find all matches
matches = re.findall(pattern, result)

# Convert F1 scores to floats and find the max
max_combination = max(matches, key=lambda x: float(x[1]))

print("Combination with the highest F1 score:")
print("Combination:", max_combination[0], "F1 Score:", max_combination[1])

Combination with the highest F1 score:
Combination: ('word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors') F1 Score: 0.6874337451825074


In [14]:
print(len(feature_combinations))
feature_combinations=feature_combinations[250:]
print(len(feature_combinations))

# Iterate over combinations, train models, and evaluate
best_f1 = 0
best_metrics = None
best_combination = None

for combo in tqdm(feature_combinations):
    # Select features for this combination
    X_train_combo = X_train[list(combo)]
    X_test_combo = X_test[list(combo)]

    # Train the Random Forest model
    rf = RandomForestClassifier()
    rf.fit(X_train_combo, y_train)

    # Predict on the test set
    predicted_labels = rf.predict(X_test_combo)

    # Evaluate the model
    metrics = getMetrics(predicted_labels, y_test)
    
    print(f"Combination: {combo}, F1 Score: {metrics['f1']}")

    # Check if this is the best model so far
    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        best_metrics = metrics
        best_combination = combo

# Return the best F1 score, the corresponding combination of features, and the full metrics
best_f1, best_combination, best_metrics

261
11


  9%|▉         | 1/11 [00:06<01:00,  6.03s/it]

Combination: ('lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4381745503956979


 18%|█▊        | 2/11 [00:11<00:52,  5.86s/it]

Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index'), F1 Score: 0.4737762093766673


...

 91%|█████████ | 10/11 [00:53<00:05,  5.38s/it]

Combination: ('sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.41296915075043794


100%|██████████| 11/11 [01:01<00:00,  5.56s/it]

Combination: ('word_count', 'sentence_count', 'lexical_diversity', 'avg_sentence_length', 'avg_word_length', 'lexical_density', 'flesch_reading_ease', 'gunning_fog_index', 'grammatical_errors'), F1 Score: 0.4820760513955631


(0.6484940938129311,
 ('word_count',
  'sentence_count',
  'avg_sentence_length',
  'avg_word_length',
  'lexical_density',
  'flesch_reading_ease',
  'gunning_fog_index',
  'grammatical_errors'),
 {'f1': 0.6484940938129311,
  'confusion_matrix': [[1864, 636], [1106, 1394]],
  'accuracy': 0.6516,
  'precision': 0.6866995073891625,
  'recall': 0.5576,
  'auc': 0.6516})

In [3]:
X_train=pd.read_csv('datasets/features_df.csv')
X_test=pd.read_csv('datasets/features_df_test.csv')
y_train=X_train['label']
y_test=X_test['label']

best_combination=['word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors']
best_f1=0.6874337451825074
print("Best Combination:", best_combination)
print("Best F1 Score:", best_f1)

Best Combination: ['word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors']
Best F1 Score: 0.6874337451825074


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

# Assuming you already have the best_combination from your previous code
X_train_best = X_train[list(best_combination)]
X_test_best = X_test[list(best_combination)]

# Hyperparameter tuning using Grid Search
param_grid = {
    # 'n_estimators': [100, 200, 300,400],
    # 'max_depth': [10, 20, 30, None],
    # 'min_samples_split': [2, 3, 5, 10],
    # 'min_samples_leaf': [1, 2, 3, 4],
    # 'max_features': ['auto', 'sqrt', 'log2'],
    'n_estimators': [100,200,300],
    'max_depth': [None],
    # 'min_samples_split': [2, 3, 5, 10],
    # 'min_samples_leaf': [1, 2, 3, 4],
    # 'max_features': ['auto', 'sqrt', 'log2'],
}

def getMetrics(predicted_labels, true_labels):
    # Ensure the labels are numpy arrays
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)

    # Compute metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')[2]
    precision = precision_score(true_labels, predicted_labels, average='binary')
    recall = recall_score(true_labels, predicted_labels, average='binary')
    auc = roc_auc_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)

    # Create a dictionary of metrics
    metrics = {
        'f1': f1,
        'confusion_matrix': cm.tolist(),
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'auc': auc,
    }

    return metrics

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_best, y_train)

# Train the final model with the best parameters
best_params = grid_search.best_params_
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train_best, y_train)

predicted_labels = final_model.predict(X_test_best)

# Get metrics for the final model
final_model_metrics = getMetrics(predicted_labels, y_test)
print(final_model_metrics)


{'f1': 0.6909230472951077, 'confusion_matrix': [[1846, 654], [888, 1612]], 'accuracy': 0.6916, 'precision': 0.7113857016769638, 'recall': 0.6448, 'auc': 0.6916}


In [6]:
with open('SavedModels/optimized-rf-features-20k.pkl', 'wb') as file:
    pickle.dump(final_model,file)

In [2]:
# Best Combination: ['word_count', 'avg_sentence_length', 'avg_word_length', 'gunning_fog_index', 'grammatical_errors']

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize the grammar checker
tool = language_tool_python.LanguageTool('en-US')

def extract_text_features(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    word_count = len(words)
    sentence_count = len(sentences)

    # Average sentence length
    avg_sentence_length = word_count / sentence_count

    # Average word length
    avg_word_length = sum(len(word) for word in words) / word_count

    # Readability scores
    fog_index = textstatistics().gunning_fog(text)

    # Grammatical errors
    matches = tool.check(text)
    grammatical_errors = len(matches)

    # Create a DataFrame
    return pd.Series({
        'word_count': word_count,
        'avg_sentence_length': avg_sentence_length,
        'avg_word_length': avg_word_length,
        'gunning_fog_index': fog_index,
        'grammatical_errors': grammatical_errors
    })

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
df= pd.read_json('datasets/subtaskA_train_monolingual.jsonl', lines=True)
test_df= pd.read_json('datasets/subtaskA_dev_monolingual.jsonl', lines=True)
df=df.sample(20000)

from tqdm import tqdm
tqdm.pandas()
features_df = df['text'].progress_apply(extract_text_features)
features_df_test = test_df['text'].progress_apply(extract_text_features)

100%|██████████| 5000/5000 [10:32<00:00,  7.91it/s]  


In [5]:
features_df['text']=df['text']
features_df_test['text']=test_df['text']
features_df['label']=df['label']
features_df_test['label']=test_df['label']

features_df.to_csv('datasets/features_df_extended.csv', index=False)
features_df_test.to_csv('datasets/features_df_test_extended.csv', index=False)